# Fuzzy String Match
* [Repo: fuzzywuzzy](https://github.com/seatgeek/fuzzywuzzy)
    * [extractOne() logic](https://github.com/seatgeek/fuzzywuzzy/blob/9e3d2fe0d8c1b195696d5fbcda78c371dd4a6b8f/fuzzywuzzy/fuzz.py#L224)
* [Repo: python-Levenshtein](https://github.com/ztane/python-Levenshtein)
    * [Core code in C](https://github.com/ztane/python-Levenshtein/blob/master/Levenshtein/_levenshtein.c)

In [97]:
from fuzzywuzzy import fuzz, process, utils

__How unwanted fuzzy matches happen? - An example from real data__

Problem: “intsig.com pdf producer” is fuzzy matched with the keyword 'PDF Pro$' with a score of 90 in extractOne().

Root Cause: 1) score calculation logic behind extractOne(); 2) less flexibility compared to regex.

Solution: exact match using regex

In [91]:
process.extractOne('intsig.com pdf producer', ['PDF Pro$'])

('PDF Pro$', 90)

In [126]:
print(fuzz.ratio('intsig.com pdf producer', 'pdf pro$')) # 45
print(fuzz.partial_ratio('intsig.com pdf producer', 'pdf pro$')) # 88

print(process.utils.full_process('intsig.com pdf producer')) # intsig com pdf producer
print(process.utils.full_process('PDF Pro$')) # pdf pro

print(fuzz.ratio('intsig com pdf producer', 'pdf pro')) # 47
print(fuzz.partial_ratio('intsig com pdf producer', 'pdf pro')) # 100 * 0.9 = 90

45
88
intsig com pdf producer
pdf pro
47
100


__What’s the math behind fuzzywuzzy? - A formula based on Levenshtein distance and input string lengths__

Theoretical Foundation: [Levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) (edit distance)<br>Fuzzywuzzy implementation: input is 2 strings, output is a ratio <br>
```
length_sum  = len(str1) + len(str2)
l_dist = Levenshtein distance(str1, str2)
ratio = (length_sum - l_dist) / length_sum
```

with variations (weighting, prioritization, ...)

In [95]:
print(fuzz.ratio("this is a test", "this is a test!"))

97


In [96]:
len_short = len("this is a test")
len_long = len("this is a test!")
l_sum = len_short + len_long
l_dist = 1
print('ratio:', (l_sum - l_dist) / l_sum)

ratio: 0.9655172413793104


__Why sometimes fuzzywuzzy score is not robust? - Rationale based on ratio formula and scoring function__

<u>Sensitive to input strings with extreme length.</u> The package tries to normalize the score by some scalers, but still not perfect or not strictly comparable. Possible improvements: 1) Given input strings, we can reverse-engineer the ratio threshold based on Levenshtein distance threshold; 2) Another way of normalization - slice the long string to multiple N-grams to roughly match the length of the short string, then perform fuzzywuzzy for each pair, tho computationally expensive.

<u>Scoring function configuration.</u> When we apply the extractOne() function to fuzzy match a string to a list of options, the scoring function is configured as a pre-determined combined approach of the normal ratio and partial ratio (=substring exact match ratio), but it might not fit all use cases. A possible improvement would be customizing the scoring function configuration based on specific use cases.

In [99]:
# sensitive to input strings
print(fuzz.ratio("12", "13")) # 50
print(fuzz.ratio("12222222222", "13")) # 15

50
15


In [105]:
# reverse engineering: derive score threshold from l_dist threshold
def threshold_converter(str1, str2, l_dist_thres = 0):
    l_sum = len(str1) + len(str2)
    return (l_sum - l_dist_thres) / l_sum

In [106]:
threshold_converter("this is a test", "this is a test!", 1)

0.9655172413793104

In [112]:
# find N-grams and do pair-wise fuzzy match
from itertools import islice, tee
def find_N_grams(s, N):
    N_grams =  list(zip(*(islice(seq, index, None) for index, seq in enumerate(tee(s, N)))))
    N_grams = [''.join(i) for i in N_grams]
    return N_grams

In [117]:
print(find_N_grams("12222222222", 2))

['12', '22', '22', '22', '22', '22', '22', '22', '22', '22']


In [119]:
def N_gram_ratio(str1, str2): # str1: shorter string; str2: longer string
    if len(str1) > len(str2):
        str1, str2 = str2, str1
    
    str2_N_grams = find_N_grams(str2, len(str1))
    
    ratios = []
    for new_str2 in str2_N_grams:
        ratio = fuzz.ratio(new_str2, str1)
        ratios.append(ratio)
    return max(ratios)

In [120]:
print(N_gram_ratio("12", "13")) # 50
print(N_gram_ratio("12222222222", "13")) # 50

50
50


# Methods

#### Simple Ratio

In [15]:
print(fuzz.ratio("this is a test", "this is a test!"))

97


In [13]:
len_short = len("this is a test")
len_long = len("this is a test!")
l_sum = len_short + len_long
l_dist = 1
print('ratio:', (l_sum - l_dist) / l_sum)

ratio: 0.9655172413793104


In [23]:
print(fuzz.ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear"))

91


In [25]:
len_short = len("fuzzy wuzzy was a bear")
len_long = len("wuzzy fuzzy was a bear")
l_sum = len_short + len_long
l_dist = 4
print('ratio:', (l_sum - l_dist) / l_sum)

ratio: 0.9090909090909091


#### Partial Ratio

In [16]:
print(fuzz.partial_ratio("this is a test", "this is a test!"))

100


In [21]:
print(fuzz.partial_ratio("123", "1234"))
print(fuzz.partial_ratio("1234", "34"))

100
100


#### Token Sort Ratio

In [30]:
fuzz.token_sort_ratio("fuzzy wuzzy was a bear", "wuzzy fuzzy was a bear")

100

#### Token Set Ratio

In [121]:
fuzz.token_set_ratio("fuzzy was a bear", "fuzzy fuzzy was a bear")

100